In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
import os
import sys
import shutil
from glob import glob

In [3]:
import numpy as np

In [4]:
try:
    lock
except:
    %cd ..
    CUR_DIR = os.getcwd()
    lock = 'Locked'
CUR_DIR

/home/ubuntu/p/fast-ai


'/home/ubuntu/p/fast-ai'

# Aims
1. Train linear model on VGG16 output to predict cat vs dogs
2. Replace final dense layer of VGG16 to predict
3. Fine-Train dense VGG16 layers
4. Fine-Train whole of VGG16 network

In [5]:
DATA_PATH = CUR_DIR + '/data/cats_dogs/'; print(DATA_PATH)
SAMPLE_PATH = DATA_PATH + 'sample/'; print(SAMPLE_PATH)
SPLIT_PATHS = ['train/','val/','test/']
CREATE_DATA_SETS = False

/home/ubuntu/p/fast-ai/data/cats_dogs/
/home/ubuntu/p/fast-ai/data/cats_dogs/sample/


# Get and setup of data structure

Setup kaggle-cli config

In [6]:
!kg config -u "dbcerigo" -c "dogs-vs-cats-redux-kernels-edition"

In [8]:
if not os.path.isdir(DATA_PATH):
    os.mkdir(DATA_PATH)
    %cd {DATA_PATH}
    !kg download
    !unzip -qn test.zip    
    !unzip -qn train.zip
    os.mkdir(DATA_PATH+'val')
if not os.path.isdir(SAMPLE_PATH):
    os.mkdir(SAMPLE_PATH)
    for path in SPLIT_PATHS:
        os.mkdir(SAMPLE_PATH+SAMPLE_PATH+path)

In [16]:
ls test

unknown/


In [18]:
%cd {DATA_PATH}
# test size
!ls -1 train | wc -l # post creating data should output 2 (cats, dogs)
# train size
!ls -1 test | wc -l # post creating data should output 1 (unknown)

/home/ubuntu/p/fast-ai/data/cats_dogs
2
1


In [19]:
!ls {DATA_PATH}
!ls sample

sample	sample_submission.csv  test  train  val
test  train  val


In [20]:
if CREATE_DATA_SETS: 
    !rm  {DATA_PATH}test.zip
    !rm  {DATA_PATH}train.zip

### FIRST, seperate off a val set (do first to avoid leaks etc.)

In [21]:
%cd train

/home/ubuntu/p/fast-ai/data/cats_dogs/train


In [22]:
im_paths = glob('*.jpg')
im_paths[:1]

[]

In [23]:
im_paths = np.random.permutation(im_paths)
if CREATE_DATA_SETS: 
    for i in range(2000): os.rename(im_paths[i], DATA_PATH+'val/'+im_paths[i])

#### Move sample parts

In [24]:
%cd {DATA_PATH}train
im_paths = glob('*.jpg') #get new paths list as don't want to ones that are in val
im_paths = np.random.permutation(im_paths)
if CREATE_DATA_SETS:
    for i in range(100): shutil.copyfile(im_paths[i], SAMPLE_PATH+'train/'+im_paths[i]) 

/home/ubuntu/p/fast-ai/data/cats_dogs/train


In [25]:
im_paths = np.random.permutation(im_paths)
if CREATE_DATA_SETS:
    for i in range(10): shutil.copyfile(im_paths[i], SAMPLE_PATH+'val/'+im_paths[i]) 

In [26]:
%cd {DATA_PATH}test
im_paths = glob('*.jpg') 
im_paths = np.random.permutation(im_paths)
if CREATE_DATA_SETS:
    for i in range(20): shutil.copyfile(im_paths[i], SAMPLE_PATH+'test/'+im_paths[i]) 
%cd {DATA_PATH}

/home/ubuntu/p/fast-ai/data/cats_dogs/test
/home/ubuntu/p/fast-ai/data/cats_dogs


In [27]:
%cd {DATA_PATH}
# test size
!ls -1 test | wc -l
# train size
!ls -1 train | wc -l
!ls -R sample

/home/ubuntu/p/fast-ai/data/cats_dogs
1
2
sample:
test  train  val

sample/test:
unknown

sample/test/unknown:
1387.jpg  2586.jpg  3801.jpg  4913.jpg	6195.jpg  8152.jpg  9078.jpg
1633.jpg  272.jpg   4402.jpg  5071.jpg	6778.jpg  8723.jpg  9290.jpg
1940.jpg  3180.jpg  4559.jpg  5241.jpg	7691.jpg  8856.jpg

sample/train:
cats  dogs

sample/train/cats:
cat.10506.jpg  cat.1852.jpg  cat.3930.jpg  cat.6134.jpg  cat.833.jpg
cat.10827.jpg  cat.2032.jpg  cat.4018.jpg  cat.6267.jpg  cat.8531.jpg
cat.11293.jpg  cat.2071.jpg  cat.4108.jpg  cat.633.jpg	 cat.8691.jpg
cat.1138.jpg   cat.2102.jpg  cat.4410.jpg  cat.6383.jpg  cat.9080.jpg
cat.11511.jpg  cat.219.jpg   cat.4459.jpg  cat.6420.jpg  cat.9152.jpg
cat.11559.jpg  cat.221.jpg   cat.4929.jpg  cat.7076.jpg  cat.9368.jpg
cat.1171.jpg   cat.247.jpg   cat.5107.jpg  cat.7203.jpg  cat.9404.jpg
cat.11733.jpg  cat.3067.jpg  cat.5122.jpg  cat.7205.jpg  cat.9750.jpg
cat.12041.jpg  cat.3120.jpg  cat.5277.jpg  cat.8216.jpg  cat.9800.jpg
cat.12131.jpg  cat.32

In [28]:
def make_move_class_files(path):
    targer_dir =path.split('/')[-2]
    assert targer_dir == 'train' or targer_dir == 'val'
    %cd {path}
    %mkdir cats
    %mkdir dogs
    %mv cat.*.jpg cats/
    %mv dog.*.jpg dogs/

In [29]:
def make_move_test_files(path):
    assert path.split('/')[-2] == 'test'
    %cd {path}
    %mkdir unknown
    %mv *.jpg unknown/

In [30]:
if CREATE_DATA_SETS:
    make_move_class_files(SAMPLE_PATH+'train/')
    make_move_class_files(SAMPLE_PATH+'val/')
    make_move_test_files(SAMPLE_PATH+'test/')
    make_move_class_files(DATA_PATH+'train/')
    make_move_class_files(DATA_PATH+'val/')
    make_move_test_files(DATA_PATH+'test/')

# 1. Train linear model on VGG16 output to predict cat vs dogs

In [31]:
import keras

Using TensorFlow backend.


### On Keras model weights
* Model weights can be considered seperate from model
* Model is just a DAG, a set of operations (or flow) on a tensor, where the actual values are not yet defined - these are the weights (the types and shapes etc. are defined)

* Key uses `keras.utils

In [ ]:
vgg = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet',
                               input_tensor=None, input_shape=None,
                               pooling=None, classes=1000)

In [ ]:
print('test')